### Lineup Optimisation with Genetic Algorithms

In [1]:
import os
import glob
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

from constants import DATA_DIR
from utils import csv_concatenate, calculate_FPTS, calculate_MAE, calculate_RMSE, cross_val, load_full_dataset

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def add_eligible_positions(df):
    pos = {'PG':[], 'SG':[], 'SF':[], 'PF':[], 'C':[], 'G':[], 'F':[]}
    
    for i in range(df.shape[0]):
        for key in pos.keys():
            if key in df.loc[i, 'Pos']:
                pos[key].append(1)
            else:
                pos[key].append(0)
    
    for key in pos.keys():
        df[key] = pos[key]

### Genetic Algorithm

In [85]:
class GeneticAlgorithm():
    #return a list of lineups with indices in df_target
    def create_random_lineups(self, df, num_lineups):
        lineups = {'PG':[], 'SG':[], 'SF':[], 'PF':[], 'C':[], 'G':[], 'F':[], 'Util':[]}
        n = df.shape[0]

        for i in range(num_lineups):
            for key in list(lineups.keys())[:7]:
                lineups[key].append(df[df[key]==1].sample(1).index[0])

            lineups['Util'].append(df.sample(1).index[0])

        df_lineups = pd.DataFrame(lineups).loc[:,['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'Util']]

        return df_lineups
    
    
    def calculate_fitness(self, df_lineups):
        fitness = []

        for i in range(df_lineups.shape[0]):
            salary = df_target.loc[df_lineups.loc[i,:].values, 'Salary'].sum()
            total_FPTS = df_target.loc[df_lineups.loc[i,:].values, 'Pred'].sum()

            #Check for duplicates
            if len(set(df_lineups.loc[i,:].values)) < 8:
                fitness.append(0)

            #Check for Salary Caps
            elif salary >= salary_cap:
                fitness.append(0)

            #Calculate the cumulative predicted FPTS
            else:
                fitness.append(total_FPTS)

        return fitness
    

    def breed(self, df_lineups):  
        df_parents = df_lineups
        positions = list(df_lineups.columns)[:-1]
        df_children = pd.DataFrame([], columns=positions)

        df_fit = df_lineups.sort_values(by='Fitness', ascending=False).reset_index(drop=True)
        parents = df_fit.head(2)

        #Prevent breeding between two same lineups
        if all(parents.loc[0,:] != parents.loc[1,:]):
            next_index = 2

            while all(parents.loc[0,:] == parents.loc[1,:]):
                rows = [0, next_index]
                parents = df_fit.loc[rows]
                parents = parents.reset_index(drop=True)
                next_index += 1

        parents = parents.loc[:, positions]
        

        for pos in positions:
            #Add parents and swap each items position by position, creating 16 lineups
            df_children = df_children.append(parents, ignore_index=True)
            df_children.loc[df_children.shape[0]-2, pos] = parents.loc[1, pos]
            df_children.loc[df_children.shape[0]-1, pos] = parents.loc[0, pos]

        #Add parents for a total of 18 lineups
        df_children = df_children.append(parents).reset_index(drop=True)
        return df_children
    
    def mutate(self, df_lineups, df_original, num_mutations):
        positions = list(df_lineups.columns)[:-1]
        df_mutants = pd.DataFrame([], columns=positions)
        df_parent = df_lineups.sort_values(by='Fitness', ascending=False).head(1).loc[:,positions]

        #Append the original
        df_mutants = df_mutants.append(df_parent)

        #Append the original and then mutate it by replacing it with a random sample for a random position
        for i in range(num_mutations):
            pos_to_swap = random.choice(positions)
            mutant_index = df_original.sample(1).index[0]

            df_mutants = df_mutants.append(df_parent, ignore_index=True)
            current_index = df_mutants.shape[0]-1
            df_mutants.loc[current_index, pos_to_swap] = df_original.loc[mutant_index, pos_to_swap]

        return df_mutants
    
    
    def evolution(self, df_random_lineups, num_mutations, num_generations):
        df_init = df_random_lineups

        for i in range(num_generations):

            df_children = self.breed(df_init)
            df_children['Fitness'] = self.calculate_fitness(df_children)

            df_mutants = self.mutate(df_children, df_random_lineups, num_mutations)
            df_mutants['Fitness'] = self.calculate_fitness(df_mutants)

            df_init = df_mutants

        df_init['Fitness'] = self.calculate_fitness(df_init.drop('Fitness', axis=1))
        df_final = df_init.sort_values(by='Fitness', ascending=False).reset_index(drop=True)

        return df_final
    
    def compounding_evolution(self, population_size=200, num_mutations=30, num_generations=30, num_compounding=5):
        optimal_lineups = []

        for i in tqdm(range(num_compounding)):
            df_random_lineups = self.create_random_lineups(df_target, population_size)
            df_random_lineups['Fitness'] = self.calculate_fitness(df_random_lineups)

            optimal = self.evolution(df_random_lineups, num_mutations, num_generations)

            optimal_lineups.append(list(optimal.loc[0,:].values[:-1].astype(int)))

        return optimal_lineups

In [86]:
ga = GeneticAlgorithm()

In [87]:
np.random.seed(23)
salary_cap = 50000

df_pred = pd.read_csv(os.path.join(DATA_DIR, 'Predictions', '20190628-19h40m-nn.csv'))

add_eligible_positions(df_pred)

In [88]:
df_cashline = pd.read_csv(os.path.join(DATA_DIR, 'Contests', 'cashline_march_2019.csv'))
df_cashline.head(5)

,Date,Type,Fee,Cashline
0,20190301,Double Up,50,335.50
1,20190302,Double Up,25,264.00
2,20190303,Double Up,25,282.25
3,20190304,Double Up,25,276.50
4,20190305,Double Up,25,280.25


In [89]:
date, pred, actual, cashline, win, earnings, = [], [], [], [], [], []

for i in tqdm(range(df_cashline.shape[0])):
    df_target = df_pred.loc[(df_pred['Salary']!=0)&(df_pred['Date']==df_cashline.loc[i, 'Date'])].reset_index(drop=True)
    
    optimal_lineups = ga.compounding_evolution()
    
    top_lineups = {'Lineup':[], 'Pred':[], 'Actual':[]}

    for j, indices in enumerate(optimal_lineups):
        top_lineups['Lineup'].append(j)
        top_lineups['Pred'].append(df_target.loc[indices, 'Pred'].sum())
        top_lineups['Actual'].append(df_target.loc[indices, 'FPTS'].sum())


    df_lineups = pd.DataFrame(top_lineups).sort_values(by='Pred', ascending=False).reset_index(drop=True)
    df_lineups = df_lineups.loc[:, ['Lineup', 'Pred', 'Actual']]

    rows = optimal_lineups[df_lineups.loc[0, 'Lineup']]
    df_best = df_target.loc[rows,:]
    display(df_best.loc[:,['Date', 'Name', 'Team', 'Pos', 'FPTS', 'Pred', 'Salary']])
    
    print('Salary:', df_target.loc[rows,'Salary'].sum())
    print('Predicted:', df_target.loc[rows,'Pred'].sum())
    print('Actual:', df_target.loc[rows,'FPTS'].sum())
    print('Cashline:', df_cashline.loc[i, 'Cashline'])
    
    pred.append(df_target.loc[rows,'Pred'].sum())
    actual.append(df_target.loc[rows,'FPTS'].sum())
    cashline.append(df_cashline.loc[i, 'Cashline'])
    
    if df_target.loc[rows, 'FPTS'].sum() >= df_cashline.loc[i, 'Cashline']:
        print('Win:', df_cashline.loc[i, 'Fee'])
        win.append(1)
        earnings.append(df_cashline.loc[i, 'Fee'])
    else:
        print('Lose:', df_cashline.loc[i, 'Fee'])
        win.append(0)
        earnings.append(-df_cashline.loc[i, 'Fee'])
    

,Date,Name,Team,Pos,FPTS,Pred,Salary
25,20190301,D'Angelo Russell,BRK,PG/SG,34.50,54.234890,8500.0
12,20190301,Bradley Beal,WAS,SG,59.75,54.058117,8700.0
17,20190301,Caris LeVert,BRK,PG/SF,30.25,32.145830,5400.0
37,20190301,Dewayne Dedmon,ATL,PF/C,45.00,35.381770,5500.0
98,20190301,Marc Gasol,TOR,C,43.00,35.717484,5200.0
100,20190301,Marcus Smart,BOS,PG/SG,24.00,27.425268,4400.0
6,20190301,Anthony Davis,NOP,PF/C,40.00,48.829304,6800.0
65,20190301,Jarrett Allen,BRK,C,17.00,33.797040,5300.0


Salary: 49800.0
Predicted: 321.589703
Actual: 293.5
Cashline: 335.5
Lose: 50


,Date,Name,Team,Pos,FPTS,Pred,Salary
30,20190302,Darren Collison,IND,PG,42.75,36.190660,5800.0
43,20190302,Derrick White,SAS,PG/SG,34.50,30.541052,4700.0
123,20190302,Luka Doncic,DAL,PG/SF,37.50,62.571583,9600.0
49,20190302,Dorian Finney-Smith,DAL,SF/PF,13.75,20.964846,3700.0
41,20190302,Derrick Favors,UTA,PF/C,54.00,33.767020,4800.0
44,20190302,Devin Booker,PHO,PG/SG,36.00,52.851414,8200.0
106,20190302,Julius Randle,NOP,PF/C,52.50,49.127728,8000.0
109,20190302,Justise Winslow,MIA,PG/SF,28.00,33.829548,5000.0


Salary: 49800.0
Predicted: 319.84385100000003
Actual: 299.0
Cashline: 264.0
Win: 25


,Date,Name,Team,Pos,FPTS,Pred,Salary
64,20190303,Jeff Teague,MIN,PG,22.25,35.924070,5500.0
17,20190303,Bradley Beal,WAS,SG,51.25,56.964306,9200.0
143,20190303,Trevor Ariza,WAS,SF/PF,28.50,36.386510,5900.0
34,20190303,Danilo Gallinari,LAC,SF/PF,34.75,41.945830,6200.0
95,20190303,Larry Nance,CLE,PF/C,27.25,38.756042,5800.0
90,20190303,Kyle Lowry,TOR,PG,54.75,43.593163,7300.0
63,20190303,Jeff Green,WAS,PF/C,20.75,31.052555,4500.0
123,20190303,Reggie Jackson,DET,PG/SG,30.75,38.091717,5500.0


...

Salary: 49900.0
Predicted: 312.031333
Actual: 275.0
Cashline: 247.5
Win: 25


,Date,Name,Team,Pos,FPTS,Pred,Salary
67,20190330,Emmanuel Mudiay,NYK,PG,35.25,39.589733,5500.0
158,20190330,Patrick Beverley,LAC,PG/SG,15.75,30.782892,4800.0
42,20190330,Damyean Dotson,NYK,SG/SF,11.75,34.261440,5400.0
110,20190330,Jonah Bolden,PHI,PF/C,43.50,22.652973,3100.0
121,20190330,Karl-Anthony Towns,MIN,C,33.75,64.958390,10400.0
49,20190330,De'Aaron Fox,SAC,PG,49.75,46.848087,7900.0
106,20190330,Joe Harris,BRK,SG/SF,29.00,27.046162,4400.0
39,20190330,D'Angelo Russell,BRK,PG/SG,53.25,54.959003,8200.0


Salary: 49700.0
Predicted: 321.09868
Actual: 272.0
Cashline: 293.25
Lose: 25


,Date,Name,Team,Pos,FPTS,Pred,Salary
74,20190331,Kemba Walker,CHO,PG,18.75,56.555240,8900.0
31,20190331,Derrick White,SAS,PG/SG,14.75,31.117617,4900.0
83,20190331,Landry Shamet,LAC,SG/SF,13.75,22.618810,3900.0
60,20190331,Jerami Grant,OKC,PF,21.75,29.772242,5100.0
66,20190331,Jonas Valanciunas,MEM,C,20.75,52.371098,7900.0
134,20190331,Will Barton,DEN,SG/SF,25.50,28.216501,4700.0
106,20190331,Paul Millsap,DEN,PF/C,37.25,37.762497,6200.0
71,20190331,Julius Randle,NOP,PF/C,28.25,53.304672,8200.0


Salary: 49800.0
Predicted: 311.718677
Actual: 180.75
Cashline: 240.5
Lose: 25


In [91]:
print(sum(win)/len(win))
print(sum(earnings))

0.3225806451612903
-285


In [92]:
date = list(set(df_cashline['Date'].values))
df_result = pd.DataFrame({'Date': date,
                          'Predicted':pred,
                          'Actual': actual,
                          'Cashline': cashline,
                          'Win':win,
                          'Earnings': earnings
                         })

df_result.loc[:,['Date', 'Predicted', 'Actual', 'Cashline', 'Win', 'Earnings']]

,Date,Predicted,Actual,Cashline,Win,Earnings
0,20190301,321.589703,293.50,335.50,0,-50
1,20190302,319.843851,299.00,264.00,1,25
2,20190303,322.714193,270.25,282.25,0,-25
3,20190304,321.288395,258.75,276.50,0,-25
4,20190305,329.631667,311.50,280.25,1,25
5,20190306,316.232911,227.00,286.50,0,-25
6,20190307,320.343421,321.75,298.00,1,25
7,20190308,314.057218,251.25,291.25,0,-25
8,20190309,314.512535,269.00,282.25,0,-25
9,20190310,314.169086,285.50,301.25,0,-25


### Baseline - random 10,000

In [93]:
df_result.loc[:,['Date', 'Predicted', 'Actual', 'Cashline', 'Win', 'Earnings']]

,Date,Predicted,Actual,Cashline,Win,Earnings
0,20190301,321.589703,293.50,335.50,0,-50
1,20190302,319.843851,299.00,264.00,1,25
2,20190303,322.714193,270.25,282.25,0,-25
3,20190304,321.288395,258.75,276.50,0,-25
4,20190305,329.631667,311.50,280.25,1,25
5,20190306,316.232911,227.00,286.50,0,-25
6,20190307,320.343421,321.75,298.00,1,25
7,20190308,314.057218,251.25,291.25,0,-25
8,20190309,314.512535,269.00,282.25,0,-25
9,20190310,314.169086,285.50,301.25,0,-25


In [96]:
df_random_lineups = ga.create_random_lineups(df_target, 200)
df_random_lineups['Fitness'] = ga.calculate_fitness(df_random_lineups)
df_random_lineups.sort_values(by='Fitness', ascending=False).head(10)

,PG,SG,SF,PF,C,G,F,Util,Fitness
56,113,20,21,7,81,4,22,39,280.204862
61,85,56,87,3,52,9,126,74,253.551206
30,39,1,4,40,66,113,76,24,250.157006
10,127,82,0,58,101,42,7,27,231.232840
51,132,123,21,97,71,84,10,33,228.119469
186,45,82,136,40,81,8,89,39,227.921983
198,117,134,73,88,66,26,4,2,225.244596
138,39,117,19,18,106,134,87,50,224.230260
111,2,76,122,14,54,134,87,113,222.681290
3,5,79,69,130,71,39,96,77,221.000854


### Visualisation

In [99]:
df_baseline = csv_concatenate(os.path.join(DATA_DIR, 'Dataframes', 'Modelling', 'Baseline'))
df_baseline['Baseline'] = calculate_FPTS(df_baseline)

In [100]:
#target_date = 20180310
# target_date = 20180326
target_date = 20190326

In [103]:
df_target = df_pred.loc[(df_pred['Salary']!=0)&(df_pred['Date']==target_date)].reset_index(drop=True)
    
optimal_lineups = ga.compounding_evolution()

top_lineups = {'Lineup':[], 'Pred':[], 'Actual':[]}

for j, indices in enumerate(optimal_lineups):
    top_lineups['Lineup'].append(j)
    top_lineups['Pred'].append(df_target.loc[indices, 'Pred'].sum())
    top_lineups['Actual'].append(df_target.loc[indices, 'FPTS'].sum())


df_lineups = pd.DataFrame(top_lineups).sort_values(by='Pred', ascending=False).reset_index(drop=True)
df_lineups = df_lineups.loc[:, ['Lineup', 'Pred', 'Actual']]

rows = optimal_lineups[df_lineups.loc[0, 'Lineup']]
df_best = df_target.loc[rows,:]
display(df_best.loc[:,['Date', 'Name', 'Team', 'Pos', 'FPTS', 'Pred', 'Salary']])

print('Salary:', df_target.loc[rows,'Salary'].sum())
print('Predicted:', df_target.loc[rows,'Pred'].sum())
print('Actual:', df_target.loc[rows,'FPTS'].sum())
print('Cashline:', int(df_cashline.loc[df_cashline['Date']==target_date, 'Cashline'].values))

,Date,Name,Team,Pos,FPTS,Pred,Salary
153,20190326,Luka Doncic,DAL,PG/SG,63.50,56.436275,8500.0
138,20190326,Kevin Huerter,ATL,SG/SF,18.25,24.093155,4200.0
144,20190326,Kyle Kuzma,LAL,SF/PF,22.00,42.260307,6700.0
61,20190326,Dirk Nowitzki,DAL,PF/C,11.00,20.170176,3300.0
15,20190326,Blake Griffin,DET,PF/C,58.75,48.617040,7800.0
189,20190326,Reggie Jackson,DET,PG/SG,32.00,28.164846,4600.0
66,20190326,Dwayne Bacon,CHO,SG/SF,43.00,25.110495,4000.0
130,20190326,Karl-Anthony Towns,MIN,C,49.75,66.691690,10700.0


Salary: 49800.0
Predicted: 311.543984
Actual: 298.25
Cashline: 265


In [104]:
print(calculate_MAE(df_best['FPTS'].values, df_best['Pred'].values))
print(calculate_RMSE(df_best['FPTS'], df_best['Pred']))

11.392084
12.762500600398203


In [105]:
df_best

,Date,Name,Team,FPTS,Pos,Salary,Pred,PG,SG,SF,PF,C,G,F
153,20190326,Luka Doncic,DAL,63.50,PG/SG,8500.0,56.436275,1,1,0,0,0,1,0
138,20190326,Kevin Huerter,ATL,18.25,SG/SF,4200.0,24.093155,0,1,1,0,0,1,1
144,20190326,Kyle Kuzma,LAL,22.00,SF/PF,6700.0,42.260307,0,0,1,1,0,0,1
61,20190326,Dirk Nowitzki,DAL,11.00,PF/C,3300.0,20.170176,0,0,0,1,1,0,1
15,20190326,Blake Griffin,DET,58.75,PF/C,7800.0,48.617040,0,0,0,1,1,0,1
189,20190326,Reggie Jackson,DET,32.00,PG/SG,4600.0,28.164846,1,1,0,0,0,1,0
66,20190326,Dwayne Bacon,CHO,43.00,SG/SF,4000.0,25.110495,0,1,1,0,0,1,1
130,20190326,Karl-Anthony Towns,MIN,49.75,C,10700.0,66.691690,0,0,0,0,1,0,0


In [106]:
Baseline = []

for i in range(df_best.shape[0]):
    df_best = df_best.reset_index(drop=True)
    Baseline.append(df_baseline.loc[(df_baseline['Name']==df_best.loc[i, 'Name'])&(df_baseline['Date']==df_best.loc[i,'Date']), 'Baseline'])

In [110]:
import plotly.plotly as py
import plotly.graph_objs as go

names = [name.split(' ')[0][:1] + '. '+ name.split(' ')[1] for name in df_best['Name']]
positions = ['(PG)', '(SG)', '(SF)','(PF)','(C)','(G)','(F)','(Utility)']

names = [names[i]+'\n'+positions[i] for i in range(8)]

trace1 = go.Bar(
    x=names,
    y=df_best['FPTS'],
    name='Actual FPTS'
)

trace2 = go.Bar(
    x=names,
    y=df_best['Pred'],
    name='Prediction'
)

trace3 = go.Bar(
    x=names,
    y=Baseline,
    name='Baseline'
)


data = [trace1, trace2, trace3]
layout = go.Layout(
    
)

layout = go.Layout(
        title = 'FPTS: Actual and Predicted',
        barmode='group',
        legend = {"x":0.85, 'y':1.3, 'borderwidth': 1},
        yaxis = {"title":"FPTS"},
    )

config={'showLink': False}

fig = go.Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename='prediction0326')
py.iplot(fig, filename='prediction0326')